In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
import pickle
import unicodedata
import regex as re
import numpy as np
from tqdm import tqdm
import unicodedata
import regex as re
import matplotlib.pyplot as plt
import random
import math
from tensorflow.keras.models import Model
import pandas as pd
from google.colab import drive
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM,Dense,Softmax

In [69]:
train = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/BACKUP DATA/TRAIN AND VALIDATION/train.csv")

validation = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/BACKUP DATA/TRAIN AND VALIDATION/validation.csv")

In [70]:
with open('/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/BACKUP DATA/tokenizer_wrong.pickle', 'rb') as handle:
    tknizer_wrng = pickle.load(handle)


with open('/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/BACKUP DATA/tokenizer_correct.pickle', 'rb') as handle:
    tknizer_corr = pickle.load(handle)

## 1.Creating embeddings for english sentences

In [71]:
!wget https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt

--2023-01-05 10:31:29--  https://www.dropbox.com/s/ddkmtqz01jc024u/glove.6B.100d.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.81.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.81.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/ddkmtqz01jc024u/glove.6B.100d.txt [following]
--2023-01-05 10:31:29--  https://www.dropbox.com/s/raw/ddkmtqz01jc024u/glove.6B.100d.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uce0d296225a3f7a402140f783ea.dl.dropboxusercontent.com/cd/0/inline/Bz8huJHkPhIaJOSbwz2QYF9WSYLNfVNJZhFMJmDjBu19xU6uKivj2lhwmhcYyzh_QT_lQTbvcCPiY7mbVUKHEtsCYsskKZxdpSjgY5v9ziMiIxLVlE5vvs_q3Y4jukapc5MlsQwW3VsSLr53PC7604BhX7ssCs5OKO5DdzNjvjbDEg/file# [following]
--2023-01-05 10:31:30--  https://uce0d296225a3f7a402140f783ea.dl.dropboxusercontent.com/cd/0/inline/Bz8huJHkPhIaJOSbwz2QYF9WSYLNfVNJZhFMJmDjBu19xU6uKivj2lhwmhcYyz

In [72]:
vocab_size_wrng = 65901
vocab_size_corr = 65713

In [73]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_matrix_enc = np.zeros((vocab_size_wrng+1, 100))
for word, i in tknizer_wrng.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_enc[i] = embedding_vector



embedding_matrix_dec = np.zeros((vocab_size_corr+1, 100))
for word, i in tknizer_corr.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_dec[i] = embedding_vector

##2. <font color='blue'>**Implement ATTENTION MODEL**</font>

### 2.1 <font color='blue'>**Encoder**</font>

In [74]:
class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns encoder-outputs,encoder_final_state_h,encoder_final_state_c
    '''

    def __init__(self,inp_vocab_size,embedding_size,lstm_size,input_length,**kwargs):

        super().__init__()
        self.vocab_size = inp_vocab_size
        self.embedding_dim = embedding_size
        self.input_length = input_length
        self.enc_units= lstm_size
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = 0,0,0
        #Initialize Embedding layer
        self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix_enc], trainable=False)
        # Intialize Encoder LSTM layer
        self.lstm = LSTM(self.enc_units, return_state=True, return_sequences=True, name="Encoder_LSTM")
        
    
    def call(self,input_sequence,states):
        '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- encoder_output, last time step's hidden and cell state
        '''
        # print("ENCODER ==> INPUT SQUENCES SHAPE :",input_sequence.shape)
        input_embedd                           = self.embedding(input_sequence)
        # print("ENCODER ==> AFTER EMBEDDING THE INPUT SHAPE :",input_embedd.shape)
        self.lstm_output, self.lstm_state_h,self.lstm_state_c = self.lstm(input_embedd)
        return self.lstm_output, self.lstm_state_h,self.lstm_state_c

      

    
    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      self.lstm_state_h = np.zeros((batch_size,self.enc_units))
      self.lstm_state_c = np.zeros((batch_size,self.enc_units))


    def get_config(self):
      config = super(Encoder,self).get_config()
      config.update({
          'vocab_size': self.vocab_size,
          'embedding_dim':self.embedding_dim,
          'input_length':self.input_length,
          'enc_units':self.enc_units,
          'lstm_output':self.lstm_output,
          'lstm_state_h':self.lstm_state_h,
          'lstm_state_c':self.lstm_state_c,
          'embedding':self.embedding,
          'lstm':self.lstm,
      })
      return config


### 2.2 <font color='blue'>**Attention**</font>

In [75]:

#defing calss attention funactions

class Attention ( tf.keras.layers.Layer ) :

    def __init__ ( self , scoring_function , att_units ) :

        super().__init__( )
        self.scoring_function  = scoring_function
        self.att_units =  att_units
        if self.scoring_function == 'dot' :
            pass
        elif scoring_function == 'general' :
            # Intialize variables for geraneal fun needed
            self.wa = Dense ( att_units )
        elif scoring_function == 'concat' :
            self.w1 = Dense ( att_units )
            self.w2 = Dense ( att_units ) 
            self.v = Dense ( 1 )

    #defining call function
    def call ( self , decoder_hidden_state , encoder_output ):

        if self.scoring_function == 'dot' :
            state = tf.expand_dims ( decoder_hidden_state , -1 )
            score = tf.matmul ( encoder_output , state )
            weights  = tf.nn.softmax ( score , axis = 1 )
            weighted_out =  encoder_output * weights
            context_vec =  tf.reduce_sum ( weighted_out , axis = 1 )
            #returing its weights
            # print(context_vec)
            # print(weights)
            return context_vec , weights
        
        elif self.scoring_function == 'general' :
            state = tf.expand_dims ( decoder_hidden_state , 2 )                                    
            score = tf.matmul ( self.wa (encoder_output ) , state )                        
            weights = tf.nn.softmax ( score , axis = 1 )  
            weighted_out = encoder_output * weights
            context_vec = tf.reduce_sum ( weighted_out , axis = 1 )
            #return contextvec and weights           
            return context_vec , weights

        elif self.scoring_function  == 'concat' :         
            state = tf.expand_dims ( decoder_hidden_state , 1 )           
            score = self.v ( tf.nn.tanh ( self.w1 ( state ) + self.w2 ( encoder_output ) ) )
            weights = tf.nn.softmax ( score , axis = 1 )
            #weighted output
            weighted_out = encoder_output * weights            
            context_vec = tf.reduce_sum ( weighted_out , axis = 1 )            
            #returing weights       
            return context_vec , weights 


    def get_config(self):
      config = super(Attention,self).get_config()
      config.update({
          'scoring_function': self.scoring_function,
          'att_units':self.att_units,
          'wa':self.wa,
          'w1':self.w1,
          'w2':self.w2,
          'v':self.v
      })
      return config


### 2.3 <font color='blue'>**OneStepDecoder**</font>

In [76]:
class One_Step_Decoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.score_fun = score_fun
      self.att_units = att_units
      #Initialize Embedding layer
      self.embedding = Embedding(input_dim=self.vocab_size, output_dim=self.embedding_dim, input_length=self.input_length,
                           mask_zero=True, name="embedding_layer_decoder", weights=[embedding_matrix_dec], trainable=False)
      #Intialize Decoder LSTM layer
      self.lstm = LSTM(self.dec_units, return_sequences=True, return_state=True, name="Encoder_LSTM") 
      #Intialize Dense layer(out_vocab_size) with activation='softmax'
      self.dense   = Dense(self.vocab_size)
      # self.dense   = Dense(self.vocab_size, activation='softmax')
      self.attention                                                         = Attention(self.score_fun,self.att_units)
      self.out_temp = []
      self.decoder_final_state_h,self.decoder_final_state_c = [],[]
      self.context_vector,self.attention_weights = [],[]
      self.concat = []
      



  def call(self,input_to_decoder, encoder_output, state_h,state_c):
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    target_embedd                                                                   = self.embedding(input_to_decoder)
    self.context_vector,self.attention_weights                                      = self.attention(state_h,encoder_output)

    self.concat = tf.concat([tf.expand_dims(self.context_vector,1),target_embedd],axis=-1)
    initial_states                                                                  = [state_h,state_c]
    decoder_output,self.decoder_final_state_h,self.decoder_final_state_c            = self.lstm(self.concat, initial_state=initial_states)
    # print("LSTM OUTPUT-->",decoder_output[:5])
    # decoder_output = tf.reshape(decoder_output,[tf.shape(decoder_output).numpy()[0],tf.shape(decoder_output).numpy()[2]])
    decoder_output = tf.reshape(decoder_output,(-1,decoder_output.shape[2]))
    output                                                                          = self.dense(decoder_output)
    # print("DENSE OUTPUT-->",output[:5])
    # print(decoder_output.shape)
    # print(output.shape)


    return output,self.decoder_final_state_h,self.decoder_final_state_c,self.attention_weights,self.context_vector
    # return np.array(output),self.decoder_final_state_h,self.decoder_final_state_c,self.attention_weights,self.context_vector

  
  def get_config(self):
      config = super(One_Step_Decoder,self).get_config()
      config.update({
          'vocab_size': self.vocab_size,
          'embedding_dim':self.embedding_dim,
          'input_length':self.input_length,
          'dec_units':self.dec_units,
          'score_fun':self.score_fun,
          'att_units':self.att_units,
          'embedding':self.embedding,
          'lstm':self.lstm,
          'dense':self.dense,
          'attention':self.attention,
          'out_temp':self.out_temp,
          'decoder_final_state_h':self.decoder_final_state_h,
          'decoder_final_state_c':self.decoder_final_state_c,
          'context_vector':self.context_vector,
          'attention_weights':self.attention_weights,
          'concat':self.concat,
      })
      return config




### 2.4 <font color='blue'>**Decoder**</font>

In [77]:
class Decoder(tf.keras.Model):
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,score_fun ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super().__init__()
      self.vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.score_fun = score_fun
      self.att_units = att_units
      #Initialize Embedding layer
      self.onestep_decoder = One_Step_Decoder(self.vocab_size, self.embedding_dim, self.input_length, self.dec_units ,self.score_fun ,self.att_units)
      self.all_outputs = tf.TensorArray(tf.float32,size=2,name="output_arrays")

      


    
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state,decoder_cell_state ):

        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        self.all_outputs = tf.TensorArray(tf.float32,size=tf.shape(input_to_decoder)[1],name="output_arrays")
      
        #Iterate till the length of the decoder input
        for timestep in range(tf.shape (input_to_decoder)[1]):
            # Call onestepdecoder for each token in decoder_input
            output,decoder_hidden_state,decoder_cell_state,_,_ = self.onestep_decoder(input_to_decoder[:,timestep:timestep+1],encoder_output,decoder_hidden_state,decoder_cell_state)
            # Store the output in tensorarray
            self.all_outputs = self.all_outputs.write(timestep,output)
        # Return the tensor array.
        # print("self.all_outputs shape-->",self.all_outputs.size())
        # print("self.all_outputs-->",self.all_outputs)
        self.all_outputs = tf.transpose(self.all_outputs.stack(),[1,0,2])
        return self.all_outputs

    
    def get_config(self):
      config = super(One_Step_Decoder,self).get_config()
      config.update({
          'vocab_size': self.vocab_size,
          'embedding_dim':self.embedding_dim,
          'input_length':self.input_length,
          'dec_units':self.dec_units,
          'score_fun':self.score_fun,
          'att_units':self.att_units,
          'onestep_decoder':self.onestep_decoder,
          'all_outputs':self.all_outputs
      })
      return config
        
        
    

### 2.5 <font color='blue'>**Encoder Decoder model**</font>

In [78]:
max_length = 70
enc_units = 256
embedding_dim = 100

In [79]:
class encoder_decoder(tf.keras.Model):
  def __init__(self,encoder_inputs_length,decoder_inputs_length, output_vocab_size,score_fun):
    #Intialize objects from encoder decoder
    super().__init__()
    self.score_fun = score_fun
    self.encoder = Encoder(inp_vocab_size=vocab_size_wrng + 1, embedding_size=embedding_dim, input_length=encoder_inputs_length, lstm_size=enc_units)#https://stackoverflow.com/questions/48479915/what-is-the-preferred-ratio-between-the-vocabulary-size-and-embedding-dimension
    self.decoder = Decoder(out_vocab_size=vocab_size_corr + 1, embedding_dim=embedding_dim, input_length=decoder_inputs_length, dec_units=enc_units,score_fun=self.score_fun,att_units=enc_units)
    self.decoder_output = []
  
  
  def call(self,data):

    input,output = data[0],data[1]
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    print("="*20, "ENCODER", "="*20)
    batch_size = 16
    enc_initial_state = self.encoder.initialize_states(batch_size)
    encoder_output, encoder_h, encoder_c = self.encoder(input,enc_initial_state)
    print("ENCODER ==> OUTPUT SHAPE",encoder_output.shape)
    print("ENCODER ==> HIDDEN STATE SHAPE",encoder_h.shape)
    # print("ENCODER ==> CELL STATE SHAPE", encoder_c.shape)
    # print("="*20, "DECODER", "="*20)
    # Decoder initial states are encoder final states, Initialize it accordingly
    dec_initial_state = [encoder_h,encoder_c]
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    self.decoder_output                             = self.decoder(output, encoder_output,encoder_h,encoder_c)
    # return the decoder output
    return self.decoder_output


  def get_config(self):
    config = super(One_Step_Decoder,self).get_config()
    config.update({
        'score_fun':self.score_fun,
        'encoder':self.encoder,
        'decoder':self.decoder,
        'decoder_output':self.decoder_output
    })
    return config



### 2.6 <font color='blue'>**Custom loss function**</font>

In [80]:
#https://www.tensorflow.org/tutorials/text/image_captioning#model
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')


def loss_function(real, pred):
    """ Custom loss function that will not consider the loss for padded zeros.
    why are we using this, can't we use simple sparse categorical crossentropy?
    Yes, you can use simple sparse categorical crossentropy as loss like we did in task-1. But in this loss function we are ignoring the loss
    for the padded zeros. i.e when the input is zero then we donot need to worry what the output is. This padded zeros are added from our end
    during preprocessing to make equal length for all the sentences.

    """
    
    
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

### 2.7 <font color='blue'>Creating data pipeline

In [81]:
class Dataset:
    def __init__(self, data, tknizer_ita, tknizer_eng, max_len):
        self.encoder_inps = data['wrong'].values
        self.decoder_inps = data['corr_inp'].values
        self.decoder_outs = data['corr_out'].values
        self.wrng_tokenizer = tknizer_wrng
        self.corr_tokenizer = tknizer_corr
        self.max_len = max_len

    def __getitem__(self, i):
        self.encoder_seq = self.wrng_tokenizer.texts_to_sequences([self.encoder_inps[i]]) # need to pass list of values
        self.decoder_inp_seq = self.corr_tokenizer.texts_to_sequences([self.decoder_inps[i]])
        self.decoder_out_seq = self.corr_tokenizer.texts_to_sequences([self.decoder_outs[i]])

        self.encoder_seq = pad_sequences(self.encoder_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_inp_seq = pad_sequences(self.decoder_inp_seq, maxlen=self.max_len, dtype='int32', padding='post')
        self.decoder_out_seq = pad_sequences(self.decoder_out_seq, maxlen=self.max_len, dtype='int32', padding='post')
        return self.encoder_seq, self.decoder_inp_seq, self.decoder_out_seq

    def __len__(self): # your model.fit_gen requires this function
        return len(self.encoder_inps)

    
class Dataloder(tf.keras.utils.Sequence):    
    def __init__(self, dataset, batch_size=1):
        self.dataset = dataset
        self.batch_size = batch_size
        self.indexes = np.arange(len(self.dataset.encoder_inps))


    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = []
        for j in range(start, stop):
            data.append(self.dataset[j])

        batch = [np.squeeze(np.stack(samples, axis=1), axis=0) for samples in zip(*data)]
        # we are creating data like ([italian, english_inp], english_out) these are already converted into seq
        return tuple([[batch[0],batch[1]],batch[2]])

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        self.indexes = np.random.permutation(self.indexes)

### 2.8 <font color='blue'>Model loading

In [82]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/ 

import tensorflow as tf
import datetime


log_dir="/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/BACKUP DATA/TENSORBOARDS/dummy/logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1, write_graph=True,write_grads=True)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [83]:
train_dataset = Dataset(train[:2], tknizer_wrng, tknizer_corr, max_length)
# test_dataset  = Dataset(validation[:2], tknizer_wrng, tknizer_corr, max_length)


train_dataloader = Dataloder(train_dataset, batch_size=2)
# test_dataloader = Dataloder(test_dataset, batch_size=2)


print(train_dataloader[0][0][0].shape, train_dataloader[0][0][1].shape, train_dataloader[0][1].shape)
# print(test_dataloader[0][0][0].shape, test_dataloader[0][0][1].shape, test_dataloader[0][1].shape)

(2, 70) (2, 70) (2, 70)


In [84]:
# full_model4 = encoder_decoder(encoder_inputs_length=max_length,decoder_inputs_length=max_length,output_vocab_size=vocab_size_corr,score_fun='dot')
# optimizer = tf.keras.optimizers.Adam(0.001,clipnorm=0.001)
# full_model4.compile(optimizer = optimizer,loss=loss_function)

In [85]:
# history = full_model4.fit(train_dataloader, epochs=25, validation_data=test_dataloader,callbacks = callbacks)
# full_model4.summary()

In [86]:
# full_model4.save_weights("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/MODELS/FINAL_ATTENTION_BLEU_81.h5")

### 2.9 <font color='blue'>Loading the weights

In [87]:
dummy = encoder_decoder(encoder_inputs_length=max_length,decoder_inputs_length=max_length,output_vocab_size=vocab_size_corr,score_fun='dot')
optimizer = tf.keras.optimizers.Adam()
dummy.compile(optimizer = optimizer,loss=loss_function)
dummy.fit(train_dataloader, epochs=1)

==================== ENCODER ====================
ENCODER ==> OUTPUT SHAPE (2, 70, 256)
ENCODER ==> HIDDEN STATE SHAPE (2, 256)
==================== ENCODER ====================
ENCODER ==> OUTPUT SHAPE (None, None, 256)
ENCODER ==> HIDDEN STATE SHAPE (None, 256)
==================== ENCODER ====================
ENCODER ==> OUTPUT SHAPE (None, None, 256)
ENCODER ==> HIDDEN STATE SHAPE (None, 256)
1/1 [==============================] - 6s 6s/step - loss: 0.8718


In [88]:
dummy.load_weights('/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/MODELS/FINAL_ATTENTION_BLEU_49.h5')

## 3. PREDICT

In [89]:
def predict(input_sentence):

  '''
  A. Given input sentence, convert the sentence into integers using tokenizer used earlier
  B. Pass the input_sequence to encoder. we get encoder_outputs, last time step hidden and cell state
  C. Initialize index of <start> as input to decoder. and encoder final states as input_states to decoder
  D. till we reach max_length of decoder or till the model predicted word <end>:
         predicted_out,state_h,state_c=model.layers[1](dec_input,states)
         pass the predicted_out to the dense layer
         update the states=[state_h,state_c]
         And get the index of the word with maximum probability of the dense layer output, using the tokenizer(word index) get the word and then store it in a string.
         Update the input_to_decoder with current predictions
  F. Return the predicted sentence
  '''
  input_sentence = input_sentence.replace("'", '')
  temp_token = tknizer_wrng.texts_to_sequences ( [ input_sentence ] )
  temp_token  = pad_sequences ( temp_token , maxlen = vocab_size_wrng , dtype = 'int32' , padding = 'post' )



  # print("=" * 30, "Inference", "=" * 30)
  initial_state =  [np.zeros((1,64)) ,np.zeros((1,64))]
  enc_output, enc_state_h, enc_state_c = dummy.layers[0](np.expand_dims(temp_token[0], 0),initial_state)
  states_values = [enc_state_h, enc_state_c]

  dec_inp = np.array ( tknizer_corr.word_index ['<start>'] ).reshape( 1 , 1 )
  attention_weights_list = []


  predicted_eng = ""
  for i in range(max_length):

    predictions , enc_state_h , enc_state_c , attention_weights , context_vector = dummy.layers[ 1 ].onestep_decoder( dec_inp, enc_output , enc_state_h, enc_state_c )
    
    word_ind  = np.argmax ( predictions,-1 )
    pred_str = list ( tknizer_corr.word_index.keys( ) ) [ int ( word_ind - 1 ) ]

    attention_weights_list.append ( attention_weights [0,:,0 ] )

    predicted_eng+= pred_str + " "

    dec_inp = word_ind.reshape(1,1)

    if(pred_str == "<end>"):
      # print(predicted_eng)
      return predicted_eng, np.array ( attention_weights_list )
  return predicted_eng, np.array ( attention_weights_list )



In [90]:
import matplotlib
def plot_attention ( attention , sentence , predicted_sentence ) :
       
    sentence  = sentence.split()
    #final sentence declaration
    sentence  = sentence 
    
    predicted_sentence =  predicted_sentence.split() + [ '<end>' ]    
    fig = plt.figure(figsize =( 10 , 10 ))
    ax = fig.add_subplot (1 , 1 , 1)
    attention = attention [:len ( predicted_sentence ), :len(sentence) ]
    
    #matrix plot with proper arguments
    ax.matshow(attention, cmap = 'viridis', vmin = 0.0)

    
    #fontsize as 14
    fontdict = {'fontsize': 14}
    #seting up axis labels argument
    ax.set_xticklabels( [''] + sentence , fontdict = fontdict , rotation = 90 )
    ax.set_yticklabels( [''] + predicted_sentence , fontdict = fontdict)
    ax.xaxis.set_major_locator ( matplotlib.ticker.MultipleLocator (1) )
    ax.yaxis.set_major_locator ( matplotlib.ticker.MultipleLocator (1) )
    
    ax.set_xlabel ( 'Input text' )   
    ax.set_ylabel ( 'Output text' )
    
    #titles
    plt.suptitle ('Attention weights')



## 4. final_fun_1(X)

In [91]:
def final_fun_1(X):

    punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''
    max_length = 70
    movie_quotes = X
    processed_movie_quotes = []
    results = []

    # Removing punctuations in string
    # Using loop + punctuation string
    for test_str in movie_quotes:
      for ele in test_str:
          if ele in punc:
              test_str = test_str.replace(ele, "")
      processed_movie_quotes.append(test_str.lower())


    for each in processed_movie_quotes:
      pred_sent,attention_weights   = predict (each)
      results.append(pred_sent.replace("<end>",""))
    

    df = pd.DataFrame(list(zip(movie_quotes, results)), columns =['INPUT', 'PREDICTED'])
    return df

In [92]:
raw_data = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/data_500k.csv")
raw_data.head()

,original,modified
0,they do not,they due not
1,they do to,they do too
2,i hope so,i hope so
3,she okay,she okay
4,let's go,let's go


In [93]:
X = raw_data["modified"].tolist()
results = final_fun_1(X[1234:1237])
results.head()

,INPUT,PREDICTED
0,yeah well ewe took the camera and put it right...,yeah well you took the camera and put it right...
1,yes you are i'm not just some reporter aye don...,yes you are im not just some reporter i dont j...
2,i not,i'm not


## 5. final_fun_2(X)

In [94]:
import warnings
warnings.filterwarnings('ignore')

import nltk.translate.bleu_score as bleu


def final_fun_2(X):

    punc = '''!()-[]{};:"\,<>./?@#$%^&*_~'''
    max_length = 70
    input = X['modified'].tolist()
    act_output = X['original'].tolist()
    processed_movie_quotes = []
    results = []

    # Removing punctuations in string
    # Using loop + punctuation string
    for test_str in input:
      for ele in test_str:
          if ele in punc:
              test_str = test_str.replace(ele, "")
      processed_movie_quotes.append(test_str.lower())


    for each in processed_movie_quotes:
      pred_sent,attention_weights   = predict (each)
      results.append(pred_sent.replace("<end>",""))
    

    
    
    
    df = pd.DataFrame(list(zip(input, act_output, results)), columns =['INPUT', 'ACTUAL_OUTPUT', 'PREDICTED_OUTPUT'])

    # print(act_output)
    # print(results)
    total_score = 0
    for i in tqdm(range(len(input))):

      reference = [act_output[i].split(),] # the original
      translation = results[i].split()# trasilated using model
      total_score += bleu.sentence_bleu(reference, translation)


    avg_score = total_score/(len(input))



    return avg_score

In [97]:
raw_data = pd.read_csv("/content/drive/MyDrive/DATA SCIENCE/CASE STUDY 2/data_500k.csv")
X = raw_data.iloc[list(range(490500,490900,5))]
final_score = final_fun_2(X)
print("\n\n\n\n\nFINAL SCORE--->",final_score)

100%|██████████| 80/80 [00:00<00:00, 7377.52it/s]






FINAL SCORE---> 0.7508849194569475
